In [1]:
# Data set: MovieLens 100k (https://grouplens.org/datasets/movielens/100k/)
import pandas as pd

ratings_df = pd.read_csv(
    "../data/ml-100k/u.data",
    sep="\t",
    names=["user_id", "movie_id", "rating", "timestamp"],
    header=None,
)


movies_df = pd.read_csv(
    "../data/ml-100k/u.item",
    sep="|",
    names=["movie_id", "movie_title", "release_date", "video_release_date", "IMDb_URL"]
    + [
        "unknown",
        "Action",
        "Adventure",
        "Animation",
        "Children's",
        "Comedy",
        "Crime",
        "Documentary",
        "Drama",
        "Fantasy",
        "Film-Noir",
        "Horror",
        "Musical",
        "Mystery",
        "Romance",
        "Sci-Fi",
        "Thriller",
        "War",
        "Western",
    ],
    header=None,
    encoding="latin-1",
)

users_df = pd.read_csv(
    "../data/ml-100k/u.user",
    sep="|",
    names=["user_id", "age", "gender", "occupation", "zip_code"],
    header=None,
)

In [2]:
ratings_df["timestamp"] = pd.to_datetime(ratings_df["timestamp"], unit="s")
ratings_df["rating_year"] = ratings_df["timestamp"].dt.year
ratings_df["rating_month"] = ratings_df["timestamp"].dt.month
ratings_df["rating_dayofweek"] = ratings_df["timestamp"].dt.day_name()

In [3]:
movies_df["release_date"] = pd.to_datetime(movies_df["release_date"], errors="coerce")
movies_df["release_year"] = movies_df["release_date"].dt.year
movies_df["release_month"] = movies_df["release_date"].dt.month
movies_df["release_dayofweek"] = movies_df["release_date"].dt.day_name()

In [4]:
genre_cols = [
    "Action",
    "Adventure",
    "Animation",
    "Children's",
    "Comedy",
    "Crime",
    "Documentary",
    "Drama",
    "Fantasy",
    "Film-Noir",
    "Horror",
    "Musical",
    "Mystery",
    "Romance",
    "Sci-Fi",
    "Thriller",
    "War",
    "Western",
]
movies_df = movies_df[
    ["movie_id", "release_year", "release_month", "release_dayofweek"] + genre_cols
]

In [5]:
merged_df = ratings_df.merge(movies_df, on="movie_id", how="left").merge(
    users_df[["user_id", "age", "gender", "occupation"]], on="user_id", how="left"
)

In [6]:
merged_df = pd.get_dummies(
    merged_df, columns=["rating_dayofweek", "release_dayofweek", "gender", "occupation"]
)

In [7]:
merged_df = merged_df.drop(columns=["timestamp"])

In [8]:
merged_df

,user_id,movie_id,rating,rating_year,rating_month,release_year,release_month,Action,Adventure,Animation,...,occupation_marketing,occupation_none,occupation_other,occupation_programmer,occupation_retired,occupation_salesman,occupation_scientist,occupation_student,occupation_technician,occupation_writer
0,196,242,3,1997,12,1997.0,1.0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,186,302,3,1998,4,1997.0,1.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,22,377,1,1997,11,1994.0,1.0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,244,51,2,1997,11,1994.0,1.0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,166,346,1,1998,2,1997.0,1.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,880,476,3,1997,11,1996.0,9.0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
99996,716,204,5,1997,11,1985.0,1.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
99997,276,1090,1,1997,9,1993.0,1.0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
99998,13,225,2,1997,12,1996.0,11.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
merged_df = merged_df.dropna()

In [10]:
user_rating_counts = merged_df.groupby("user_id").size()
eligible_users = user_rating_counts[user_rating_counts >= 30].index
test_user_ids = pd.Series(eligible_users).sample(80, random_state=42)
test_df = merged_df[merged_df["user_id"].isin(test_user_ids)]

In [ ]:
# test_user_ids = merged_df['user_id'].sample(5, random_state=42)
# test_df = merged_df[merged_df['user_id'].isin(test_user_ids)]

In [11]:
train_df = merged_df[~merged_df["user_id"].isin(test_user_ids)]

In [15]:
from sklearn.preprocessing import StandardScaler

numeric_cols = [
    "age",
    "rating_year",
    "rating_month",
    "release_year",
    "release_month",
    "movie_id",
]
scaler = StandardScaler()
train_df[numeric_cols] = scaler.fit_transform(train_df[numeric_cols])
test_df[numeric_cols] = scaler.transform(test_df[numeric_cols])

c:\Users\Moakher\miniconda3\envs\bandits\lib\site-packages\pandas\core\frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)
c:\Users\Moakher\miniconda3\envs\bandits\lib\site-packages\pandas\core\frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)


In [16]:
train_df

,user_id,movie_id,rating,rating_year,rating_month,release_year,release_month,Action,Adventure,Animation,...,occupation_marketing,occupation_none,occupation_other,occupation_programmer,occupation_retired,occupation_salesman,occupation_scientist,occupation_student,occupation_technician,occupation_writer
0,196,-0.554181,3,-0.971752,1.23436,0.637907,-0.525610,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,186,-0.373027,3,1.029070,-0.61884,0.637907,-0.525610,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,22,-0.146585,1,-0.971752,1.00271,0.425619,-0.525610,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,244,-1.130855,2,-0.971752,1.00271,0.425619,-0.525610,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,166,-0.240181,1,1.029070,-1.08214,0.637907,-0.525610,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99994,378,-1.049336,3,-0.971752,1.00271,0.354857,-0.525610,0,1,0,...,0,0,0,0,0,0,0,1,0,0
99995,880,0.152319,3,-0.971752,1.00271,0.567144,2.034498,0,0,0,...,0,0,0,0,0,0,0,1,0,0
99996,716,-0.668912,5,-0.971752,1.00271,-0.211244,-0.525610,0,0,0,...,0,0,0,0,0,0,0,0,0,0
99997,276,2.006129,1,-0.971752,0.53941,0.354857,-0.525610,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [17]:
import numpy as np

unique_users = train_df["user_id"].unique()
T = len(unique_users)
n = train_df.groupby("user_id").size().max()
d = train_df.shape[1] - 2

X = np.zeros((T, n, d))
Y = np.zeros((T, n))

for i, user_id in enumerate(unique_users):
    user_data = train_df[train_df["user_id"] == user_id]
    user_features = user_data.drop(columns=["user_id", "rating"]).to_numpy()
    user_ratings = user_data["rating"].to_numpy()

    X[i, : len(user_features), :] = user_features
    Y[i, : len(user_ratings)] = user_ratings

In [ ]:
X_test = np.zeros((T, n, d))
Y_test = np.zeros((T, n))

for i, user_id in enumerate(unique_users):
    user_data = test_df[test_df["user_id"] == user_id]
    user_features = user_data.drop(columns=["user_id", "rating"]).to_numpy()
    user_ratings = user_data["rating"].to_numpy()

    X_test[i, : len(user_features), :] = user_features
    Y_test[i, : len(user_ratings)] = user_ratings

In [25]:
test_df["user_id"].nunique()

80

In [27]:
X_test = []
Y_test = []
unique_users = test_df["user_id"].unique()


for user_id in unique_users:
    user_data = test_df[test_df["user_id"] == user_id]
    user_features = user_data.drop(columns=["user_id", "rating"]).to_numpy()
    user_ratings = user_data["rating"].to_numpy()

    X_test.append(user_features)
    Y_test.append(user_ratings)

In [19]:
np.save("../data/X_ml_80test.npy", X)
np.save("../data/Y_ml_80test.npy", Y)

In [ ]:
import pickle

with open("../data/X_test_80test.pkl", "wb") as f:
    pickle.dump(X_test, f)
with open("../data/Y_test_80test.pkl", "wb") as f:
    pickle.dump(Y_test, f)